### Make sure run this notebook in electra repo folder after git clone,

```bash
git clone https://github.com/google-research/electra.git
cd electra
jupyter notebook
```

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
# !wget https://storage.googleapis.com/electra-data/electra_base.zip
# !unzip electra_base.zip
!ls electra_base/

checkpoint			  electra_base.index  vocab.txt
electra_base.data-00000-of-00001  electra_base.meta


In [3]:
import configure_finetuning
from util import training_utils

hparams = {'model_size': 'base', 'vocab_size': 30522}
config = configure_finetuning.FinetuningConfig('electra-base', './electra_base/', **hparams)
bert_config = training_utils.get_bert_config(config)

bert_config.__dict__

{'vocab_size': 30522,
 'hidden_size': 768,
 'num_hidden_layers': 12,
 'num_attention_heads': 12,
 'hidden_act': 'gelu',
 'intermediate_size': 3072,
 'hidden_dropout_prob': 0.1,
 'attention_probs_dropout_prob': 0.1,
 'max_position_embeddings': 512,
 'type_vocab_size': 2,
 'initializer_range': 0.02}

In [4]:
import tensorflow as tf
from model import modeling
from model import optimization

In [5]:
# !wget https://raw.githubusercontent.com/huseinzol05/NLP-Models-Tensorflow/master/text-classification/utils.py
# !wget https://raw.githubusercontent.com/huseinzol05/NLP-Models-Tensorflow/master/text-classification/data.zip
# !unzip data.zip

In [6]:
from utils import *
from sklearn.model_selection import train_test_split

In [7]:
trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['negative', 'positive']
10662
10662


In [8]:
MAX_SEQ_LENGTH = 100

In [9]:
from model import tokenization

tokenizer = tokenization.FullTokenizer(
        vocab_file='electra_base/vocab.txt',
        do_lower_case=True)

In [10]:
from tqdm import tqdm

input_ids, input_masks, segment_ids = [], [], []

for text in tqdm(trainset.data):
    tokens_a = tokenizer.tokenize(text.lower())
    if len(tokens_a) > MAX_SEQ_LENGTH - 2:
        tokens_a = tokens_a[:(MAX_SEQ_LENGTH - 2)]
    tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
    segment_id = [0] * len(tokens)
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_id)
    padding = [0] * (MAX_SEQ_LENGTH - len(input_id))
    input_id += padding
    input_mask += padding
    segment_id += padding
    
    input_ids.append(input_id)
    input_masks.append(input_mask)
    segment_ids.append(segment_id)

100%|██████████| 10662/10662 [00:02<00:00, 3926.33it/s]


In [11]:
from finetune import task_builder

In [12]:
config

In [13]:
tasks = task_builder.get_tasks(config)
tasks[0].name

'chunk'

In [14]:
batch_size = 32
epoch = 10
num_train_steps = int(len(input_ids) / batch_size * epoch)

class Model:
    def __init__(
        self,
        dimension_output,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.segment_ids = tf.placeholder(tf.int32, [None, None])
        self.input_masks = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        
        model = modeling.BertModel(
            bert_config=bert_config,
            is_training=False,
            input_ids=self.X,
            input_mask=self.input_masks,
            token_type_ids=self.segment_ids,
            use_one_hot_embeddings=False)
        
        output_layer = model.get_pooled_output()
        
        with tf.variable_scope("task_specific/classify"):
            self.logits = tf.layers.dense(output_layer, dimension_output)
        
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        self.optimizer = optimization.create_optimizer(
              self.cost, config.learning_rate, num_train_steps,
              weight_decay_rate=config.weight_decay_rate,
              use_tpu=config.use_tpu,
              warmup_proportion=config.warmup_proportion,
              layerwise_lr_decay_power=config.layerwise_lr_decay,
              n_transformer_layers=bert_config.num_hidden_layers
          )
        
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [15]:
dimension_output = 2

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
)

sess.run(tf.global_variables_initializer())

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [16]:
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'electra')
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, 'electra_base/electra_base')

INFO:tensorflow:Restoring parameters from electra_base/electra_base


In [17]:
train_input_ids, test_input_ids, train_input_masks, test_input_masks, train_segment_ids, test_segment_ids, train_Y, test_Y = train_test_split(
    input_ids, input_masks, segment_ids, trainset.target, test_size = 0.2
)

In [18]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, len(train_input_ids), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_input_ids))
        batch_x = train_input_ids[i: index]
        batch_masks = train_input_masks[i: index]
        batch_segment = train_segment_ids[i: index]
        batch_y = train_Y[i: index]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.segment_ids: batch_segment,
                model.input_masks: batch_masks
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    pbar = tqdm(range(0, len(test_input_ids), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_input_ids))
        batch_x = test_input_ids[i: index]
        batch_masks = test_input_masks[i: index]
        batch_segment = test_segment_ids[i: index]
        batch_y = test_Y[i: index]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.segment_ids: batch_segment,
                model.input_masks: batch_masks
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= len(train_input_ids) / batch_size
    train_acc /= len(train_input_ids) / batch_size
    test_loss /= len(test_input_ids) / batch_size
    test_acc /= len(test_input_ids) / batch_size

    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 0/267 [00:00<?, ?it/s]

epoch: 0, pass acc: 0.000000, current acc: 0.815016
time taken: 70.1569435596466
epoch: 0, training loss: 0.457605, training acc: 0.783341, valid loss: 0.414334, valid acc: 0.815016



train minibatch loop:   0%|          | 0/267 [00:00<?, ?it/s]

epoch: 1, pass acc: 0.815016, current acc: 0.823901
time taken: 65.72079563140869
epoch: 1, training loss: 0.265715, training acc: 0.898595, valid loss: 0.624250, valid acc: 0.823901



train minibatch loop:   0%|          | 0/267 [00:00<?, ?it/s]

...

time taken: 65.80612897872925
epoch: 3, training loss: 0.031535, training acc: 0.993551, valid loss: 1.316631, valid acc: 0.825330



train minibatch loop:  83%|████████▎ | 221/267 [00:51<00:10,  4.35it/s, accuracy=0.969, cost=0.083] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

train minibatch loop:   0%|          | 0/267 [00:00<?, ?it/s]

time taken: 66.04801869392395
epoch: 7, training loss: 0.000043, training acc: 1.001759, valid loss: 1.700782, valid acc: 0.833523



test minibatch loop: 100%|██████████| 67/67 [00:04<00:00, 14.69it/s, accuracy=0.81, cost=1.3]   

time taken: 66.06311511993408
epoch: 8, training loss: 0.000004, training acc: 1.001759, valid loss: 1.716736, valid acc: 0.833523

break epoch:9

